In [1]:
from env import env
import pandas as pd


In [2]:
df = pd.read_csv('results/bike-results/result.csv')

### Get od pair

In [3]:
import random as rd

In [11]:
# try later: https://geobgu.xyz/py/shapely.html

lat = df['station_lat'].to_list()
lng = df['station_lng'].to_list()

rd.seed(31)
x = rd.sample(range(0,2191),2)

originCoord = (lat[x[0]],lng[x[0]])
origin = f'{originCoord[0]},{originCoord[1]}'

destinationCoord = (lat[x[1]],lng[x[1]])
destination = f'{destinationCoord[0]},{destinationCoord[1]}'

print(origin)
print(destination)

41.9106,-87.6494
41.76,-87.58


### Here requests

In [14]:
from haversine import haversine, Unit
import urllib.request
import json, sys

In [ ]:
"""
def closest(coord): # need refactor

    data = pd.read_csv()
    dist = sys.float_info.max
    waypoint = ()
    for ind,row in data.iterrows():
        dist_test = distance(coord[0],coord[1],row['station_lat'],row['longitude']) # use haversine
        
        if dist_test < dist:
            dist = dist_test
            waypoint = (row['latitude'],row['longitude'])
    return waypoint
"""

In [23]:
# get closest bike share point on coordinate
def closest(coord: list): #### need refactor!!
    data = pd.read_csv('results/bike-results/result.csv')
    dist = sys.float_info.max
    waypoint = ()
    for ind,row in data.iterrows():
        distTest = haversine((float(coord[0]),float(coord[1])),(row['station_lat'],row['station_lng']))
        if distTest < dist:
            dist = distTest
            waypoint = (row['station_lat'],row['station_lng'])
    return waypoint

#### Full route:

In [22]:
def here_full_request(origin:str,destination:str,maxAlternatives=0):
    query = f'destination={destination}&origin={origin}'
    hereKey = env('HERE_API_KEY')

     # Car co2 and fuel consumption link: https://developer.here.com/documentation/routing-api/dev_guide/topics/use-cases/calculate-fuel-consumption.html

    mode = ['car','bus','bicycle']
    fuelConsumption = [
        '&fuel[type]=diesel&fuel[freeflowspeedtable]=0,0,60,0.1,80,0.12&fuel[trafficspeedtable]=0,0,60,0.1,80,0.12&fuel[additionalconsumption]=1.8&fuel[ascent]=0.1',
        '&fuel[type]=diesel&fuel[freeflowspeedtable]=0,0,60,0.1,80,0.12&fuel[trafficspeedtable]=0,0,60,0.1,80,0.12&fuel[additionalconsumption]=1.8&fuel[ascent]=0.1', # add fuel consumption parameters for bus
        '' 
    ]

    responses = []
    
    for i in range(len(mode)):
        if mode[i] == 'bicycle':
            newOrigin = closest(origin.split(','))
            newDestination = closest(destination.split(','))
            query = f'destination={newDestination[0]},{newDestination[1]}&origin={newOrigin[0]},{newOrigin[1]}'
        url = 'https://router.hereapi.com/v8/routes?transportMode=%s%s&%s&maxAlternatives=%d&return=summary,polyline&apikey=%s' % (mode[i],fuelConsumption[i],query,maxAlternatives,hereKey)
        response = urllib.request.urlopen(url).read().decode('utf-8')
        responses.append(json.loads(response))
    return responses
#full_responses = here_full_request(origin,destination)

In [24]:
full_responses = here_full_request(origin,destination)

#### Car hybrid:

In [ ]:
def here_car_request(origin,destination,midpoint,maxAlternatives=0): # unfinished
    query = [
        f'destination={midpoint}&origin={origin}', # head
        f'destination={destination}&origin={midpoint}' # tail
    ]
    hereKey = env('HERE_API_KEY')

    # car-bus-walk
    # car-bike-walk

    modeOne = ['car','bus']
    modeTwo = ['car','bicycle']

    fuelConsumptionOne = [ # car - bus
        '&fuel[type]=diesel&fuel[freeflowspeedtable]=0,0,60,0.1,80,0.12&fuel[trafficspeedtable]=0,0,60,0.1,80,0.12&fuel[additionalconsumption]=1.8&fuel[ascent]=0.1',
        '&fuel[type]=diesel&fuel[freeflowspeedtable]=0,0,60,0.1,80,0.12&fuel[trafficspeedtable]=0,0,60,0.1,80,0.12&fuel[additionalconsumption]=1.8&fuel[ascent]=0.1' # add fuel consumption parameters for bus
    ]

    fuelConsumptionTwo = [ # car - bicycle
        '&fuel[type]=diesel&fuel[freeflowspeedtable]=0,0,60,0.1,80,0.12&fuel[trafficspeedtable]=0,0,60,0.1,80,0.12&fuel[additionalconsumption]=1.8&fuel[ascent]=0.1',
        ''
    ]

    for i in range(len(query)):
        urlOne = 'https://router.hereapi.com/v8/routes?transportMode=%s%s&%s&maxAlternatives=%d&return=summary,polyline&apikey=%s' % (modeOne[i],fuelConsumptionOne[i],query[i],maxAlternatives,hereKey)
        urltwo = 'https://router.hereapi.com/v8/routes?transportMode=%s%s&%s&maxAlternatives=%d&return=summary,polyline&apikey=%s' % (modeTwo[i],fuelConsumptionTwo[i],query[i],maxAlternatives,hereKey)

    

#### Transit Hybrid:

In [ ]:
def here_transit_request(origin,destination,midpoint,maxAlternatives=0):
    query = [
        f'destination={midpoint}&origin={origin}', # head
        f'destination={destination}&origin={midpoint}' # tail
    ]
    hereKey = env('HERE_API_KEY')

#### Bicycle hybrid

In [ ]:
def here_bicycle_request(origin,destination,midpoint,maxAlternatives=0):
    query = [
        f'destination={midpoint}&origin={origin}', # head
        f'destination={destination}&origin={midpoint}' # tail
    ]
    hereKey = env('HERE_API_KEY')

#### TomTom routing test with car:

In [ ]:
#query = f'{origin}:{destination}'
#tomtomkey = env('TOM_TOM_KEY')

#url='https://api.tomtom.com/routing/1/calculateRoute/%s/json?&key=%s&sectionType=traffic&instructionsType=coded&maxAlternatives=%d&traffic=true&travelMode=car' % (query, tomtomkey, maxAlternatives)

#### Here routing test with car:

In [6]:
query = f'destination={destination}&origin={origin}'
hereKey = env('HERE_API_KEY')
fuelConsumption = 'transportMode=car&fuel[type]=diesel&fuel[freeflowspeedtable]=0,0,60,0.1,80,0.12&fuel[trafficspeedtable]=0,0,60,0.1,80,0.12&fuel[additionalconsumption]=1.8&fuel[ascent]=0.1'

url = 'https://router.hereapi.com/v8/routes?%s&%s&return=summary,polyline&apikey=%s' % (fuelConsumption,query,hereKey)

hereResponse = urllib.request.urlopen(url).read().decode('utf-8')
hereResponse = json.loads(hereResponse)

### Recover polyline coordinates from request 

In [5]:
import flexpolyline as fp

In [5]:
# to be determined
#colors = {
#    'pedestrian':'blue',
#    'bus':'yellow',
#    'bicycle':'green'
#}

# add colors to different sections of route depending on ransport modal

def coord_reader(response):
    latList = []
    lngList = []

    for i in response['routes'][0]['sections']:
        try:
            polyline = fp.decode(i['polyline'])

            for coord in polyline:
                latList.append(coord[0])
                lngList.append(coord[1])
        
        except:
            pass

        latList.append(i['arrival']['place']['location']['lat'])
        lngList.append(i['arrival']['place']['location']['lng'])
    
    return latList,lngList

In [7]:
latList,lngList = coord_reader(hereResponse)

### Route midpoint

In [8]:
#Route midpoint
middle = delta = hereResponse['routes'][0]['sections'][0]['summary']['length']/2
zipList = list(zip(latList,lngList))
before = after = None
dist = 0

for i in range(len(zipList)-1):
    dist += haversine(zipList[i],zipList[i+1], unit=Unit.METERS)
    sub = abs(middle-dist)
    if sub < delta:
        delta = sub
        before = zipList[i+1]
    if sub > delta:
        after = zipList[i+1]
        break

midpointCoord = ((before[0]+after[0])/2,(before[1]+after[1])/2)
midpoint = f'{midpointCoord[0]},{midpointCoord[1]}'

print(f'route midpoint for multimodal: {midpointCoord}')

route midpoint for multimodal: (41.842255, -87.61019)


### Route map print

In [ ]:
from bs4 import BeautifulSoup as bs
import gmplot

In [11]:
def map_print(fileName,originCoord,latList,lngList,midpointCoord,pointMarker=False):

    outputMap = gmplot.GoogleMapPlotter(originCoord[0],originCoord[1], 12)

    outputMap.plot(latList, lngList, edge_width=3)
    mapUrl = f'results/map_plot/{fileName}.html'

    if pointMarker:
        outputMap.marker(midpointCoord[0],midpointCoord[1],color='green')

    outputMap.draw(mapUrl)

    # adding the api key into html file
    page = open(mapUrl)
    soup = bs(page.read(),"html.parser")
    srcKey  = f"https://maps.googleapis.com/maps/api/js?key={env('GOOGLE_MAPS_KEY')}&libraries=visualization&sensor=true_or_false"
    line = soup.find('script',type="text/javascript")
    line['src'] = srcKey

    with open(mapUrl, "wb") as f_output:
        f_output.write(soup.prettify("utf-8"))  

In [14]:
map_print('Here_map',originCoord,latList,lngList,midpointCoord)